- [Reference video](https://www.bilibili.com/video/BV1Y44y1b7ke/)
- State-transition equation:
$$x(k+1)=\underbrace{A}_{n \times n}\underbrace{x(k)}_{n \times 1}+\underbrace{B}_{n \times p}\underbrace{u(k)}_{p \times 1}$$
- Prediction horizon function:
$$
X(k)=Mx(k) + CU(k)
$$
- In this function(N is horizon)
$$
X(k) =
\underbrace{
\begin{bmatrix}
x(k|k) \\
x(k+1|k) \\
\vdots \\
x(k+i|k) \\
\vdots \\
x(k+N|k) \\
\end{bmatrix}
}_{n(N+1) \times 1}
\quad \\
U(k)=
\underbrace{
\begin{bmatrix}
u(k|k) \\
u(k+1|k) \\
\vdots \\
u(k+i|k) \\
\vdots \\
u(k+N-1|k) \\
\end{bmatrix}
}_{Np \times 1}
\quad \\
M =
\underbrace{
\begin{bmatrix}
I_{n \times n} \\
A_{n \times n} \\
A_2 \\
\vdots \\
A_N \\
\end{bmatrix}
}_{n(N+1) \times n}
\quad \\
C =
\underbrace{
\begin{bmatrix}
0 & 0 & \cdots & 0 \\
\vdots & \vdots & \cdots & \vdots \\
0 & 0 & & 0 \\
B & 0 & \cdots & 0 \\
AB & B & \cdots & 0 \\
\vdots & \vdots & \ddots & 0 \\
A^{N-1}B & A^{N-2}B & \cdots & B \\
\end{bmatrix}
}_{n(N+1) \times Np}
$$


- Cost function:
$$
J = (x(k)-x^*)^TG(x(k)-x^*)+U(k)^THU(k)+2(x(k)-x^*)^TEU(k)
$$
- In this funciton
$$
G = M^T\overline{Q}M \\
\quad \\
E = M^T\overline{Q}C \\
\quad \\
H = C^T\overline{Q}C + \overline{R} \\
$$

- $Q(error),R(input)$ are adjustment matrixs
$$
\overline{Q} =
\begin{bmatrix}
Q & \cdots &  \\
\vdots & Q & \vdots \\
  & \cdots & F \\
\end{bmatrix}
\quad \\
\overline{R} =
\begin{bmatrix}
R & \cdots &  \\
\vdots & \ddots & \vdots \\
  & \cdots & R \\
\end{bmatrix}
$$

In [1]:
import sympy


def solve(mat: sympy.Matrix, symbol: sympy.MatrixSymbol, val):
    return mat.subs(symbol, val)


def M(N: int, A: sympy.Matrix):
    return sympy.BlockMatrix([[sympy.Matrix(A**i)] for i in range(N + 1)]).as_explicit()


def G(N: int, A: sympy.Matrix, Q_BAR: sympy.Matrix):
    return M(N, A).T * Q_BAR * M(N, A)


def C(N: int, A: sympy.Matrix, B: sympy.Matrix):
    n, p = (A * B).shape
    res_up = sympy.BlockMatrix([[sympy.zeros(n, p) for i in range(N)]]).as_explicit()
    res_down = sympy.BlockMatrix(
        [
            [
                A ** (N - j) * (A ** (N - i)).inv() * B if i >= j else sympy.zeros(n, p)
                for j in range(N)
            ]
            for i in range(N)
        ]
    ).as_explicit()
    return sympy.BlockMatrix([[res_up], [res_down]]).as_explicit()


def Q_BAR(N: int, Q: sympy.Matrix, F: sympy.Matrix):
    return sympy.BlockDiagMatrix(*([Q for _ in range(N)] + [F])).as_explicit()


def R_BAR(N: int, R: sympy.Matrix):
    return sympy.BlockDiagMatrix(*([R for _ in range(N)])).as_explicit()


def G(M: sympy.Matrix, Q_BAR: sympy.Matrix):
    return M.T * Q_BAR * M


def E(M: sympy.Matrix, Q_BAR: sympy.Matrix, C: sympy.Matrix):
    return M.T * Q_BAR * C


def H(C: sympy.Matrix, Q_BAR: sympy.Matrix, R_BAR: sympy.Matrix):
    return C.T * Q_BAR * C + R_BAR


def MPC_GEH(
    N: int,
    A: sympy.Matrix,
    B: sympy.Matrix,
    Q: sympy.Matrix,
    F: sympy.Matrix,
    R: sympy.Matrix,
):
    return (
        G(M(N, A), Q_BAR(N, Q, F)),
        E(M(N, A), Q_BAR(N, Q, F), C(N, A, B)),
        H(C(N, A, B), Q_BAR(N, Q, F), R_BAR(N, R)),
    )


def J(
    G: sympy.Matrix,
    E: sympy.Matrix,
    H: sympy.Matrix,
    x_k: sympy.Matrix,
    x_target: sympy.Matrix,
    U_K: sympy.MatrixSymbol,
):
    res1 = (x_k - x_target).T * G * (x_k - x_target)
    res2 = U_K.T * H * U_K
    res3 = 2 * (x_k - x_target).T * E * U_K
    return res1 + res2 + res3

- ex($n,p=2,2$):
$$
\begin{bmatrix}
x_1(k+1) \\
x_2(k+1) \\
\end{bmatrix}
=  
\begin{bmatrix}
1  & 0.1 \\
-1 & 2 \\
\end{bmatrix}
\begin{bmatrix}
x_1(k) \\
x_2(k) \\
\end{bmatrix}
+
\begin{bmatrix}
0.2 & 1 \\
0.5 & 2 \\
\end{bmatrix}u(k)
$$

In [2]:
# State,input dimensions
n, p = 2, 2
# Prediction horizon
N = 3

"""
# Also use symbolic calculations to view the results
# All results are symbols rather than numbers
# Example:
A_SYM = sympy.MatrixSymbol("A", n, n)
B_SYM = sympy.MatrixSymbol("B", n, p)
Q_SYM = sympy.MatrixSymbol("Q", n, n)
F_SYM = sympy.MatrixSymbol("F", n, n)
R_SYM = sympy.MatrixSymbol("R", p, p)
G_SYM, E_SYM, H_SYM = MPC_GEH(N, A_SYM, B_SYM, Q_SYM, F_SYM, R_SYM)
"""
# State transition matrix
A_VAL = sympy.Matrix([[1, 0.1], [-1, 2]])
B_VAL = sympy.Matrix([[0.2, 1], [0.5, 2]])
# Adjustment matrix
Q_VAL = sympy.Matrix([[1, 0], [0, 1]])
F_VAL = sympy.Matrix([[1, 0], [0, 1]])
R_VAL = sympy.Matrix([[0.1, 0], [0, 0.1]])

G_VAL, E_VAL, H_VAL = MPC_GEH(N, A_VAL, B_VAL, Q_VAL, F_VAL, R_VAL)

# x_k is now state
x_k_SYM = sympy.MatrixSymbol("x_k", n, 1)
# x_target is target state
x_target_SYM = sympy.MatrixSymbol("x^*", n, 1)
# U_K is the mpc result
U_K_SYM = sympy.MatrixSymbol("U", N * p, 1)

# Minimize J
J_SYM = J(G_VAL, E_VAL, H_VAL, x_k_SYM, x_target_SYM, U_K_SYM)

display(A_VAL, B_VAL, Q_VAL, F_VAL, R_VAL, G_VAL, E_VAL, H_VAL, J_SYM)

Matrix([
[ 1, 0.1],
[-1,   2]])

Matrix([
[0.2, 1],
[0.5, 2]])

Matrix([
[1, 0],
[0, 1]])

Matrix([
[1, 0],
[0, 1]])

Matrix([
[0.1,   0],
[  0, 0.1]])

Matrix([
[  60.78, -64.666],
[-64.666, 77.0361]])

Matrix([
[-11.592, -41.14,  -6.69, -25.08, -3.33, -13.2],
[14.5677, 53.195, 8.1825, 31.428, 3.888, 15.69]])

Matrix([
[3.0239, 10.875, 1.6125, 6.296, 0.741, 3.03],
[10.875,  40.83,  5.955,  23.4,   2.7, 11.1],
[1.6125,  5.955, 1.0925,   3.9,  0.45, 1.85],
[ 6.296,   23.4,    3.9, 15.54,  1.74,  7.2],
[ 0.741,    2.7,   0.45,  1.74,  0.39,  1.2],
[  3.03,   11.1,   1.85,   7.2,   1.2,  5.1]])

(-2*x^*.T + 2*x_k.T)*Matrix([
[-11.592, -41.14,  -6.69, -25.08, -3.33, -13.2],
[14.5677, 53.195, 8.1825, 31.428, 3.888, 15.69]])*U + (-x^*.T + x_k.T)*Matrix([
[  60.78, -64.666],
[-64.666, 77.0361]])*(-x^* + x_k) + U.T*Matrix([
[3.0239, 10.875, 1.6125, 6.296, 0.741, 3.03],
[10.875,  40.83,  5.955,  23.4,   2.7, 11.1],
[1.6125,  5.955, 1.0925,   3.9,  0.45, 1.85],
[ 6.296,   23.4,    3.9, 15.54,  1.74,  7.2],
[ 0.741,    2.7,   0.45,  1.74,  0.39,  1.2],
[  3.03,   11.1,   1.85,   7.2,   1.2,  5.1]])*U